# Binary Addition
> 8-bit Binary addition using Recurrent Neural Networks. 

### Rules of Binary Addition
![](addition_rules.jpg)

> Out network tries to learn these rules

### Feed Forward Neural Network for Binary Addition

![](RNN.jpg)

In [6]:
def add_binary_nums(x,y):
        x=bin(x)[2:]
        y=bin(y)[2:]
        max_len = max(len(x), len(y))
        flag = False
        x = x.zfill(max_len)
        y = y.zfill(max_len)

        result = ''
        carry = 0

        for i in range(max_len-1,-1,-1):
            r = carry
            r += 1 if x[i] == '1' else 0
            r += 1 if y[i] == '1' else 0
            result = ('1' if r % 2 == 1 else '0') + result
            carry = 0 if r < 2 else 1       

        if carry !=0 : 
            flag=True
            result = '1' + result

        return flag

In [7]:
has_carry = 0
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

In [8]:
import copy
import numpy as np

In [9]:
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output
def sigmoid_derivative(output):
    return output*(1-output)

In [10]:
correct = 0
int2binary = {}
binary_dim = 8

In [11]:
largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)],dtype=np.uint8).T,axis=1)
for i in range(largest_number):
    int2binary[i] = binary[i]

In [12]:
alpha = 0.1
input_dim = 2
hidden_dim = 16
output_dim = 1

In [13]:
Wx = 2*np.random.random((input_dim,hidden_dim)) - 1
Wy = 2*np.random.random((hidden_dim,output_dim)) - 1
Wr = 2*np.random.random((hidden_dim,hidden_dim)) - 1

Wx_update = np.zeros_like(Wx)
Wy_update = np.zeros_like(Wy)
Wr_update = np.zeros_like(Wr)

In [14]:
correct = 0
has_carry = 0

for j in range(75000):
    np.random.seed(2)
    a_int = np.random.randint(largest_number/2) 
    a = int2binary[a_int]

    b_int = np.random.randint(largest_number/2)
    b = int2binary[b_int]

    c_int = a_int + b_int
    c = int2binary[c_int]
    
    pred = np.zeros_like(c)

    overallError = 0
    
    dyts = list()
    hts = list()
    hts.append(np.zeros(hidden_dim))
    
    for position in range(binary_dim):
        
        X = np.array([[a[binary_dim - position - 1],b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T

        ht = sigmoid(np.dot(X,Wx) + np.dot(hts[-1],Wr))

        yt = sigmoid(np.dot(ht,Wy))

        yt_error = y - yt
        dyts.append((yt_error)*sigmoid_derivative(yt))
        overallError += np.abs(yt_error[0])
    
        pred[binary_dim - position - 1] = np.round(yt[0][0])
        
        hts.append(copy.deepcopy(ht))
    
    future_dht = np.zeros(hidden_dim)
    
    for position in range(binary_dim):
        
        X = np.array([[a[position],b[position]]])
        ht = hts[-position-1]
        prev_ht = hts[-position-2]
        dyt = dyts[-position-1]
        dht = (future_dht.dot(Wr.T) + dyt.dot(Wy.T)) * sigmoid_derivative(ht)
        Wy_update += np.atleast_2d(ht).T.dot(dyt)
        Wr_update += np.atleast_2d(prev_ht).T.dot(dht)
        Wx_update += X.T.dot(dht)
        
        future_dht = dht
    

    Wx += Wx_update * alpha
    Wy += Wy_update * alpha
    Wr += Wr_update * alpha    

    Wx_update *= 0
    Wy_update *= 0
    Wr_update *= 0
    
    if np.array_equal(pred.T,c):
        correct+=1
    else:
        if add_binary_nums(a_int,b_int):
            has_carry+=1

### Accuracy

In [15]:
(correct/75000)*100

99.14933333333333

### Number of samples that have an error and involve a carry term in their binary addition

In [16]:
has_carry

0

### Very Small Gradients

In [19]:
dht

array([[ 2.21659339e-05,  1.44377290e-05, -6.42845105e-06,
         3.57158139e-06,  1.46810873e-05, -4.04056167e-06,
        -5.76981202e-06, -3.40344995e-06,  1.14527439e-05,
         3.33269394e-07, -1.91304835e-06,  1.12321051e-05,
        -1.11633949e-05,  1.22563900e-05,  2.86657948e-06,
        -5.72074485e-06]])